*Copyright (c) Microsoft Corporation. All rights reserved.*

*Licensed under the MIT License.*

# Text Classification of MultiNLI Sentences using PyTorch Transformers

In [4]:
# Import packages
import os
import sys
import json 
import pandas as pd
import numpy as np
import torch

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

from utils_nlp.dataset.multinli import load_pandas_df
from utils_nlp.models.transformers.sequence_classification import Processor, SequenceClassifier 
from utils_nlp.common.timer import Timer

In [5]:
# Set QUICK_RUN = True to run the notebook on a small subset of data and a smaller number of epochs
QUICK_RUN = False

## Introduction 
## [TODO] - Modify for the final model
This notebook fine-tunes and evaluates a pretrained [XLNet](https://arxiv.org/pdf/1906.08237.pdf) model on a subset of the [MultiNLI](https://www.nyu.edu/projects/bowman/multinli/) dataset.

We use a [sequence classifier](../../utils_nlp/models/bert/sequence_classification.py) that wraps [Hugging Face's PyTorch implementation](https://github.com/huggingface/pytorch-pretrained-BERT) of Google's [BERT](https://github.com/google-research/bert).

In [7]:
TRAIN_DATA_FRACTION = 1
TEST_DATA_FRACTION = 1
NUM_EPOCHS = 3

if QUICK_RUN:
    TRAIN_DATA_FRACTION = 0.01
    TEST_DATA_FRACTION = 0.01
    NUM_EPOCHS = 1

BATCH_SIZE = 32 if torch.cuda.is_available() else 8
DATA_FOLDER = "./temp"
MODEL_CACHE_DIR = "./temp"
TO_LOWER = True
MAX_LEN = 150
BATCH_SIZE_PRED = 512
TRAIN_SIZE = 0.6
LABEL_COL = "genre"
TEXT_COL = "sentence1" 

### [TODO] - Remove Workflow overview

```
model_name = SequenceClassifier.list_supported_models()[0]
num_labels = len(label_encoder.classes_)
processor = Processor(model_name=model_name, cache_dir=temp_dir)
ds = processor.preprocess(text_train, labels_train, max_len=max_len)
classifier = SequenceClassifier(
    model_name=model_name, num_labels=num_labels, cache_dir=temp_dir
)
classifier.fit(ds, device="cuda", num_epochs=1, batch_size=32, num_gpus=None)
```